In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [7]:
url = "https://raw.githubusercontent.com/juanmendiz/ML-JUAN/master/data/Train.csv"
X_train = pd.read_csv(url)

url = "https://raw.githubusercontent.com/juanmendiz/ML-JUAN/master/data/Test.csv"
X_test = pd.read_csv(url)

# url = "https://raw.githubusercontent.com/juanmendiz/ML-JUAN/master/data/y_test.csv"
# y_test = pd.read_csv(url)

url = "https://raw.githubusercontent.com/juanmendiz/ML-JUAN/master/data/y_train.csv"
y_train = pd.read_csv(url)

# Regresion Logistica

In [7]:
# me da fallo, porq es para una clasificacion binaria el roc_auc_score?

rl = LogisticRegression(max_iter=1000)
rl.fit(X_train,y_train)

y_pred = rl.predict(X_test)

# medir_auc('Regresion Logistica', y_pred)
print("MSE :",mean_squared_error(y_test, y_pred))
print("MSLR :",mean_squared_error(y_test, y_pred))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

c:\Users\jamen\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE : 0.654721911160881
MSLR : 0.654721911160881
MAPE: 0.23581560283687944


# KNN

In [8]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)

# medir_auc('KNN ',y_pred)
print("MSE :",mean_squared_error(y_test, y_pred))
print("MSLR :",mean_squared_error(y_test, y_pred))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

c:\Users\jamen\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


MSE : 0.7637178051511758
MSLR : 0.7637178051511758
MAPE: 0.23096304591265399


# Arbol de decision

In [ ]:

tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)

y_pred = tree.predict(X_test)

# medir_auc('Arbol de decision' ,y_pred)
print("MSE :",mean_squared_error(y_test, y_pred))
print("MSLR :",mean_squared_log_error(y_test, y_pred))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))